In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from Models.grid_search_utils import GridSearch as Gs
from pandas import DataFrame

import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
    

In [ ]:
LGBMRanker = Gs.load_model(name="../Models/BlackBox/lmart/saved_models/LGBMRanker")
FIGSReg = Gs.load_model(name="../Models/GlassBox/figs/saved_models/FIGSReg")
EBMReg = Gs.load_model(name="../Models/GlassBox/ebm/saved_models/EBMReg")
test_set = pd.read_csv("../outputs/scores/0_dataset_ts.csv")

In [ ]:
job = test_set[test_set["qId"]==84].copy()

output_lgmb = LGBMRanker.predict(job.iloc[:,5:].to_numpy())
output_ebm = EBMReg.predict(job.iloc[:,5:].to_numpy())
output_figs = FIGSReg.predict(job.iloc[:,5:].to_numpy())

rank_w_score = pd.DataFrame(job["w_score"].values, index=job.index, columns=["pred"])
rank_w_score = rank_w_score.sort_values(by="pred", ascending=False).head(10)
rank_w_score["rank"] = rank_w_score["pred"].rank(method="first")
rank_w_score["ax"] = 0


rank_lgmb = pd.DataFrame(output_lgmb, index=job.index, columns=["pred"])
rank_lgmb = rank_lgmb.sort_values(by="pred",ascending=False).head(10)
rank_lgmb["rank"] = rank_lgmb["pred"].rank(method="first")
rank_lgmb["ax"] = 1

rank_ebm = pd.DataFrame(output_ebm, index=job.index, columns=["pred"])
rank_ebm = rank_ebm.sort_values(by="pred",ascending=False).head(10)
rank_ebm["rank"] = rank_ebm["pred"].rank(method="first")
rank_ebm["ax"] = 2

rank_figs = pd.DataFrame(output_figs, index=job.index, columns=["pred"])
rank_figs = rank_figs.sort_values(by="pred",ascending=False).head(10)
rank_figs["rank"] = rank_figs["pred"].rank(method="first")
rank_figs["ax"] = 3

In [ ]:
rank_ebm

In [ ]:
job

In [ ]:
sns.set_style("darkgrid")

_, axs = plt.subplots(figsize=(9, 5))

color = ["Red","Black","Blue","Green"]
for i, dt in enumerate([rank_w_score,rank_lgmb,rank_ebm,rank_figs]):
    for kId in dt.index:
        y = dt.loc[kId, "rank"] - 0.2
        x = dt.loc[kId, "ax"]
        axs.text(x, y, kId, horizontalalignment='center', size=14, color=color[i])
        
axs.set_xticks([-1, 0, 1, 2, 3, 4],["","Ground truth","LGBM Ranker","EBM","FIGS",""])
axs.set_yticks([*range(1,12)],[str(i)+"°" for i in range(10,0,-1)]+[""])
axs.tick_params(axis='both', labelsize=15)
axs.set_xlabel("")
axs.set_ylabel("Ranking position",fontsize=16)
axs.set_title("How to change the ranking by different models", fontsize=20)
axs.grid(axis='x')
plt.show()